### スコアリングフェーズにおけるデータ処理（解決編）

##### 課題把握編のおさらい
前編にて、スコアリングデータのone-hotエンコーディング後に、以下の問題が生じる得ることを確認しました。<br>
そこでこの問題が生じないよう、コードを補強していくことにしましょう。
- モデルデータにないカラムが生成される可能性
- モデルデータにあったカラムが消える可能性
- データ型の違いが理由で上記を生じさせてしまう可能性

##### データ前処理時の前提
- 私達はどのデータが数値変数で、どのデータがカテゴリ変数かを事前に把握している
- そして今回のカテゴリ変数は以下6変数とする
 - Dependents, Gender, Married, Education, Self_Employed, Property_Area

##### モデル用データの前処理
最初の補強は、ファイル読込段階で<b>カテゴリ変数をobject型として明記すること</b>です。

In [ ]:
# カテゴリ変数をリストで設定
ohe_columns = ['Dependents',
               'Gender',
               'Married',
               'Education',
               'Self_Employed',
               'Property_Area']

# カテゴリ変数をobject型で読み込むための準備
my_dtype = {'Dependents':object,
            'Gender':object,
            'Married':object,
            'Education':object,
            'Self_Employed':object,
            'Property_Area':object}

# 表示オプションの変更
import pandas as pd
pd.options.display.max_columns = 50

In [ ]:
# Loan screening model data for classification 
df = pd.read_csv('./data/av_loan_u6lujuX_CVtuZ9i.csv',
                 header=0,
                 dtype=my_dtype)

X  = df.iloc[:,:-1]            # 最終列以前を特徴量
ID = X.iloc[:,[0]]             # 第0列はPK（Loan_ID）なのでIDとしてセット
X  = X.drop('Loan_ID', axis=1) # Loan_IDは特徴ベクトルから削除
y  = df.iloc[:,-1]             # 最終列を正解データ

# check the shape
print('---------------------------------')
print('Raw shape: (%i,%i)' %df.shape)
print('X shape: (%i,%i)' %X.shape)
print('---------------------------------')
print(X.dtypes)

# ローン審査でNOとなったサンプルを1（正例）として変換
class_mapping = {'N':1, 'Y':0}
y = y.map(class_mapping)

##### モデル用データの前処理：カテゴリ変数の数量化と欠損対応
ここに変更はありません。

In [ ]:
X_ohe = pd.get_dummies(X,
                       dummy_na=True,
                       columns=ohe_columns)

print('X_ohe shape:(%i,%i)' % X_ohe.shape)
X_ohe.head()

##### モデル用データの前処理：数値変数の欠損値対応
ここも変更はありません。

In [ ]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer()
imp.fit(X_ohe)

X_ohe_columns = X_ohe.columns.values
X_ohe = pd.DataFrame(imp.transform(X_ohe), columns=X_ohe_columns)

X_ohe.head()

##### モデル用データの前処理：次元圧縮（特徴量選択）
最後に、RFEによる特徴量選択を実施します。<br>ここも変更ありません。

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

selector = RFE(RandomForestClassifier(n_estimators=100, random_state=1),
               n_features_to_select=10,
               step=.05)

selector.fit(X_ohe,y)

X_fin = X_ohe.loc[:, X_ohe_columns[selector.support_]]
print('X_fin shape:(%i,%i)' % X_fin.shape)
X_fin.head()

以上でモデリング段階の処理が終わりです。

##### スコア用データの前処理：データの読み込み
さて、ここからがテスト用データへの処理です。<br>
ここでも明示的に型を指定しましょう。

In [ ]:
# Loan screening test data for classification 
df_s = pd.read_csv('./data/av_loan_test_Y3wMUE5_7gLdaTN.csv',
                   header=0,
                   dtype=my_dtype)

ID_s = df_s.iloc[:,[0]]            # 第0列はPK（Loan_ID）なのでIDとしてセット
X_s  = df_s.drop('Loan_ID',axis=1) # Loan_IDは特徴ベクトルから削除

# check the shape
print('---------------------------------')
print('Raw shape: (%i,%i)' %df_s.shape)
print('X shape: (%i,%i)' %X_s.shape)
print('---------------------------------')
print(X_s.dtypes)
ID_s.join(X_s).head(3)

##### スコア用データの前処理：カテゴリ変数の数量化と欠損対応
カテゴリ変数のデータ型をobject型に統一し、one-hotエンコーディングをしているので、`Dependents`変数関連の表記のブレが解消しています。

In [ ]:
X_ohe_s = pd.get_dummies(X_s,
                         dummy_na=True,
                         columns=ohe_columns)
print('X_ohe_s shape:(%i,%i)' % X_ohe_s.shape)
X_ohe_s.head(3)

##### スコア用データの前処理：one-hotエンコーディング後のデータ整合チェック
さて、この時点の特徴量リストを再度比較してみます。

In [ ]:
cols_model = set(X_ohe.columns.values)
cols_score = set(X_ohe_s.columns.values)

diff1 = cols_model - cols_score
print('モデルのみに存在する項目: %s' % diff1)

diff2 = cols_score - cols_model
print('スコアのみに存在する項目: %s' % diff2)

##### 残されたデータ不整合
データ型の違いによる不一致は消え、以下の違いだけが残りました。
- Dependents_3+はスコアリングデータには存在しない
- Gender_Unknownはスコアリングデータで新たに登場した

##### 不整合解消のための基本指針
以下が基本方針となります。暗記ではなく、なぜそうすべきなのかを考えてみましょう。
- モデル用にはあるが、スコア用に存在しない変数は復活させる
- スコア用データにあるが、モデル用に存在しない変数は削除する

##### 不整合解消のためのtips：Pandasデータフレームのconcat関数
上記指針実現の一つの方法が、データフレームを縦に結合するconcat処理です。<br>一旦サンプルデータからはなれ、concat関数の動作を確認しましょう。

In [ ]:
# カラム構成が同じデータフレームの結合
df1 = pd.DataFrame([[1,2,3]], columns=['c1','c2','c3'])
df2 = pd.DataFrame([[3,2,1]], columns=['c1','c2','c3'])
df_all = pd.concat([df1, df2])
df_all

In [ ]:
# カラム構成が異なるデータフレームの結合
df3 = pd.DataFrame([[0,1,2,3]],columns=['c0','c1','c3','c4'])
df_all = pd.concat([df_all, df3])
df_all

##### concat関数の動作
- c0とc4は、元々のデータフレーム（df_all）に存在しなかった変数なので、スコア用データに初めて登場した変数と言えます。
 - よって、対応は削除となります。
- 一方、c2はモデル用データにはあったがスコア用データにはなかった変数と言えます。
 - よって、対応は変数の復活です。
 - かつ、このような不一致はカテゴリ変数でのみ生じるので、ゼロ補完が妥当です。

##### スコア用データの前処理：one-hotエンコーディング後のデータ不整合の解消
それではサンプルデータに戻ります。<br>
モデリング時点のone-hotエンコーディング後のカラム構成は`X_ohe_columns`でした。<br>
そこで、このカラム構成だけを持った（データ部分は持たない）データフレームを作ります。

In [ ]:
df_cols_m = pd.DataFrame(None,
                         columns=X_ohe_columns,
                         dtype=float)
display(df_cols_m)

上記に対し、スコアリング時点のone-hotエンコーディング後のデータを結合します。<br>
実行し以下を確認しましょう。
- `Dependents_3+`が復活（ただし、値は欠損状態）
- `Gender_Unknown`はまだ存在

In [ ]:
X_ohe_s2 = pd.concat([df_cols_m, X_ohe_s])
print(X_ohe_s2.shape)
display(X_ohe_s2.head(3))

次に、スコアリングデータのみに登場する変数`Gender_Unknown`を削除しましょう。

In [ ]:
set_Xm = set(X_ohe.columns.values)
set_Xs = set(X_ohe_s.columns.values)

X_ohe_s3 = X_ohe_s2.drop(list(set_Xs-set_Xm),axis=1)

print(X_ohe_s3.shape)
display(X_ohe_s3.head(3))

次に、スコアリングでは登場しなかったデータ項目をゼロ埋めします。<br>
実行し以下を確認しましょう。
- 変数`Depend_3+`がゼロで埋まっていること

In [ ]:
X_ohe_s3.loc[:,list(set_Xm-set_Xs)] = X_ohe_s3.loc[:,list(set_Xm-set_Xs)].fillna(0,axis=1)
X_ohe_s3.head(3)

##### スコア用データの前処理：スコア用データの変数の並び順の制御
最後に、モデリング時点のデータ項目の並び順を明示的に担保しましょう。<br>
以下の通り、`reindex`関数を使うことで並び順を制御できます。

In [ ]:
test = pd.DataFrame([[1,2,3]], columns=['c1','c2','c3'])
display(test)
test = test.reindex(['c2','c3','c1'], axis=1)
display(test)

ここでは、モデリング時点のone-hotエンコーディング後の並び順に制御します。

In [ ]:
X_ohe_s3 = X_ohe_s3.reindex(X_ohe.columns.values,axis=1)
X_ohe_s3.head(3)

##### スコア用データの前処理：数値変数の欠損値対応
ここまで整合させてようやく、学習済みSimpleImputerインスタンスが適用可能となります。

In [ ]:
print('欠損個数（数値変数の欠損補完前）',X_ohe_s3.isnull().sum().sum())
X_ohe_s4 = pd.DataFrame(imp.transform(X_ohe_s3),columns=X_ohe_columns)
print('欠損個数（数値変数の欠損補完後）',X_ohe_s4.isnull().sum().sum())

RFEによって選択された変数の位置はsupport_属性から取得できたので、<br>スコアリングデータの特徴量の最終形は以下のようになります。

##### スコア用データの前処理：特徴量選択
学習済みのRFEクラスのインスタンス（本ファイルではselector）を使って、選択された特徴量のインデックスを指定します。

In [ ]:
X_fin_s = X_ohe_s4.loc[:, X_ohe_columns[selector.support_]]
print(X_fin_s.shape)
X_fin_s.head(3)

以上で、スコアリング段階におけるデータの前処理が終了です。

<b>[やってみよう]</b> 本ファイルのタイトルをおさらいし、黒ペンで、モデル用データの前処理、スコア用データの前処理の手順をノートに一度書き出そう。色ペンに持ち替え、各処理の内容を書き加えていこう。